In [2]:
import numpy as np
from scipy.special import factorial
from scipy.optimize import bisect
from scipy.stats import norm

# Poisson Error

Let $\lambda_u$ and $\lambda_l$ be the upper limit and lower limit of confidence level $CL$ for the observed number counts $N$. Then, based on Poisson statistics, these parameters are defined by
\begin{align*}
    \sum_{x = 0}^{N} \frac{\lambda_u^x}{x!} e^{-\lambda_u} &= 1 - CL \\
    \sum_{x = 0}^{N-1} \frac{\lambda_l^x}{x!} e^{-\lambda_l} &= CL, (N \neq 0)
\end{align*}
The lower limit for $N = 0$ is $\lambda_l = 0$. Then, the uncertainty/error in $N$, expressed as
\begin{align*}
    N^{+\Delta N_u}_{-\Delta N_l}
\end{align*}
is given by
\begin{align*}
    \Delta N_u &= \lambda_u - N \\
    \Delta N_l &= N - \lambda_l
\end{align*}
However, for large $N$, the expressions can be estimated by Gaussian statistics as
\begin{align*}
    \lambda_u &= N + S \sqrt{N} \\
    \lambda_l &= N - S \sqrt{N}
\end{align*}
and consequently
\begin{align*}
    \Delta N_u &= S \sqrt{N} \\
    \Delta N_l &= S \sqrt{N}
\end{align*}
where $S$ is the number of standard deviations for the Gaussian distribution, $\sigma$. 

Note: The confidence level can be converted back and forth to the number of standard deviations $S$ for a Gaussian distribution using the following relation
\begin{align*}
    CL &= \mathrm{CDF}(S) \\
    S &= \mathrm{PPF}(CL)
\end{align*}

where $\mathrm{CDF}$ and $\mathrm{PPF}$ are the cumulative distribution function and percent point function, respectively.

Reference: [Gehrels, N. 1986, ApJ, 303, 336](https://ui.adsabs.harvard.edu/abs/1986ApJ...303..336G/abstract)

In [3]:
def poisson_error(N,CL=None,sigma=None):
    # initialize poisson functions
    def poisson(l,x):
        return np.power(l,x)*np.exp(-l)/factorial(x)
    def upper_gehrels(l,N,CL):
        x = np.arange(N+1)
        p = poisson(l,x)
        return np.sum(p) - (1 - CL)
    def lower_gehrels(l,N,CL):
        x = np.arange(N)
        p = poisson(l,x)
        return np.sum(p) - CL
    
    # initialize CL and sigma
    if sigma:
        CL = norm.cdf(sigma)
    if CL:
        sigma = norm.ppf(CL)
    
    if N <= 100: # bisection on exact functions
        upper = bisect(upper_gehrels,0,N*10 + 10,args=(N,CL)) - N
        lower = N - bisect(lower_gehrels,0,N*10 + 10,args=(N,CL)) if N > 0 else 0
    else: # gaussian approximation
        upper = sigma*np.sqrt(N)
        lower = sigma*np.sqrt(N)
    return lower, upper

In [4]:
for i in [0,1,5,10,20,50,100,200,500,1000]:
    print(f'The Poisson limits for N = {i} are:',poisson_error(i,sigma=1))

The Poisson limits for N = 0 are: (0, 1.8410216450081407)
The Poisson limits for N = 1 are: (0.8272462209763489, 2.299526559115975)
The Poisson limits for N = 5 are: (2.1596911444083844, 3.3824726521453385)
The Poisson limits for N = 10 are: (3.10869443936312, 4.26694976101075)
The Poisson limits for N = 20 are: (4.434447982213001, 5.546519229511304)
The Poisson limits for N = 50 are: (7.047336825856867, 8.11822460647388)
The Poisson limits for N = 100 are: (9.983254822884845, 11.033360941150093)
The Poisson limits for N = 200 are: (14.142135623730951, 14.142135623730951)
The Poisson limits for N = 500 are: (22.360679774997898, 22.360679774997898)
The Poisson limits for N = 1000 are: (31.622776601683793, 31.622776601683793)
